In [9]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

# Financial Statements Standardizer

## Goal

Even when adhering to the US-GAAP standard, financial statements among different companies or even across different years of the same company are often not directly comparable.

Let's examine the balance sheet to illustrate a couple of problems that can arise:

- There are over 3000 different tags that could be used in a balance sheet, even though a balance sheet typically only has about 30-40 rows.
- Some tags have a similar meaning; for instance, the position "Total assets" can be tagged with "Assets," but sometimes, the tag "AssetsNet" is also used.
- Sometimes not all major positions are presented. For instance, normally, you expect Liabilities, LiabilitiesCurrent, and LiabilitiesNoncurrent to appear in the balance sheet. However, in some reports, only Liabilities, Liabilities, and only detailed positions of LiabilitiesNoncurrent are listed, with no total position for LiabilitiesCurrent. Sometimes even the total position for liabilities is missing.

The Standardizer processes the data and produces comparable statements that contain the main positions of a certain financial statement. For example, the balance sheet standardizer produces reports with values for Assets, AssetsCurrent, AssetsNoncurrent, Liabilities, LiabilitiesCurrent, LiabilitiesNoncurrent, Equity, as well as a few other positions that are not always present.

To achieve this, the standardizer uses a **simple rule framework** that lets you define rules acting on the data. In the context of the balance sheet, a few rules include:
- If there is an AssetsNet tag but no Assets tag, copy the value from AssetsNet to Assets.
- If two of the tags Assets, AssetsCurrent, AssetsNoncurrent are present, calculate the missing one by applying the formula Assets = AssetsCurrent + AssetsNoncurrent.
- If the LiabilitiesNoncurrent tag is missing, sum up any existing detail tags of LiabilitiesNoncurrent and store the sum in the LiabilitiesNoncurrent tag.

Since calculations are involved, which under certain circumstances could be incorrect or problematic, **any action is logged**. Therefore, if a specific rule was applied for a certain report/financial statement, it is logged. With that information, a user can trace how many rules and which rules were applied to which tags of a particular report.

As mentioned, applying certain rules could lead under certain circumstances to incorrect results or interpretations. Moreover, the input data could also be incorrect or essential information could be missing from the dataset altogether. Therefore, **validation rules** can be defined and are applied at the end of processing the data. In the case of the balance sheet, a few examples of validation rules are Assets = AssetsCurrent + AssetsNoncurrent, Liabilities = LiabilitiesCurrent + LiabilitiesNoncurrent, Assets = Liabilities + Equity. These validation checks are applied for every financial statement, and the results are presented with a relative error and a categorized error (category 0 = exact match, category 1 = less than 1%, category 5 = less than 5%, category 10 = less than 10%, category 100 = greater than 10%). For instance, if you want to use the data to train an ML model, you might want to choose only statements with all checks below category 5.


## Main Process

The main process comprises four key steps:

1. **Preprocessing**
    1. Removal of unused tags: Based on predefined rules, all tags not utilized by these rules are eliminated.
    1. Deduplication: Occasionally, values for certain tags or even entire sets of tags in financial statements may be duplicated. These redundant entries need to be removed.
    1. Inversion of negated values: The sign of values marked as negated is inverted.
    1. Table pivoting: Currently, each tag and its corresponding value have their own row in the dataset. The goal is to transform this structure so that each tag has its own column.
    1. Filtering for main statements: Some financial reports contain multiple tables attributed to a specific financial statement. This step aims to retain only the main statement.
    1. Application of preprocess rules: These rules are designed to rectify errors in the data. For example, there may be reports where the tags for Assets and AssetsNoncurrent are interchanged, causing the value of Assets to be tagged as AssetsNoncurrent and vice versa. Preprocess rules help correct such errors.
    1. Preparation of log dataframes.

2. **Main Processing**<br> This step applies the main rules, following the order in which they are defined. The entire rule tree can be executed multiple times, as applying a rule at the end of the tree could calculate a previously absent tag that can then be used to calculate another value in the next iteration.

3. **Postprocessing**<br> Postprocess rules are applied in this step. Their primary purpose is to refine the results, such as setting values to zero.

4. **Finalizing**<br> Validation rules are applied, and summary logs are generated.


## Preparing the Example

In order to explain the details of the standardizer, we apply the balance sheet standardizer on the reports of the year 2022.

In [6]:
# first, create a collector which collects all reports for 2022
from secfsdstools.d_container.databagmodel import JoinedDataBag
from secfsdstools.e_collector.zipcollecting import ZipCollector
from secfsdstools.u_usecases.bulk_loading import default_postloadfilter

collector = ZipCollector.get_zip_by_names(names=["2022q1.zip", "2022q2.zip", "2022q3.zip", "2022q4.zip"], 
                                          forms_filter=["10-K", "10-Q"],                                        
                                          stmt_filter=["BS"], post_load_filter=default_postloadfilter)

joined_bag: JoinedDataBag = collector.collect().join()
print("number of loaded reports: ", len(joined_bag.sub_df))

2024-01-11 06:42:25,242 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2024-01-11 06:42:25,262 [INFO] parallelexecution      items to process: 4
2024-01-11 06:42:51,966 [INFO] parallelexecution      commited chunk: 0


number of loaded reports:  26357


**Note:** <br> we only load the 10-K and 10-Q reports. We also filter directly just for balance sheet information. We also apply the default_postloadfilter, which includes ReportPeriodRawFilter, MainCoregRawFilter, OfficialTagsOnlyRawFilter, and USDOnlyRawFilter. You definitely should apply the ReportPeriodRawFilter and the USDOnlyRawFilter. The standardizer should work without applying the MainCoregRawFilter and therefore also standardizing the statements for subsidiaries, however, I didn't test it.

## Using the standardizer

The standardizer implements the presenter interface, so you could pass it as parameter to the `present` method of the `JoinedDataBag` instance, but you could also call the `process` method of the standardizer and providing the `pre_num_df` of the `JoinedDataBag` as input.

In [8]:
from secfsdstools.f_standardize.bs_standardize import BalanceSheetStandardizer

standardizer = BalanceSheetStandardizer()
standardized_bs_df = joined_bag.present(standardizer)

In [10]:
standardized_bs_df[:10]

tag,adsh,coreg,report,ddate,uom,Assets,AssetsCurrent,AssetsNoncurrent,Liabilities,LiabilitiesCurrent,LiabilitiesNoncurrent,HolderEquity,TemporaryEquity,RedeemableEquity,Equity,LiabilitiesAndEquity,Cash,RetainedEarnings,AdditionalPaidInCapital,TreasuryStockValue,AssetsCheck_error,AssetsCheck_cat,LiabilitiesCheck_error,LiabilitiesCheck_cat,EquityCheck_error,EquityCheck_cat,AssetsLiaEquCheck_error,AssetsLiaEquCheck_cat
0,0000002178-22-000033,,3,20211231,USD,3.747030e+08,2.732100e+08,1.014930e+08,2.143170e+08,1.860110e+08,2.830600e+07,1.603860e+08,0.0,0.0,1.603860e+08,3.747030e+08,9.782500e+07,1.430400e+08,16913000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000002178-22-000046,,2,20220331,USD,4.701170e+08,3.709720e+08,9.914500e+07,3.045960e+08,2.769790e+08,2.761700e+07,1.655210e+08,0.0,0.0,1.655210e+08,4.701170e+08,9.929500e+07,1.480660e+08,17020000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000002178-22-000066,,2,20220630,USD,5.046230e+08,4.080060e+08,9.661700e+07,3.371710e+08,3.109720e+08,2.619900e+07,1.674520e+08,0.0,0.0,1.674520e+08,5.046230e+08,6.772800e+07,1.494750e+08,17541000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0000002178-22-000089,,2,20220930,USD,4.621230e+08,3.266470e+08,1.354760e+08,2.928720e+08,2.452310e+08,4.764100e+07,1.692510e+08,0.0,0.0,1.692510e+08,4.621230e+08,8.651000e+07,1.505950e+08,18218000.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000002488-22-000016,,5,20211231,USD,1.241900e+10,8.583000e+09,3.836000e+09,4.922000e+09,4.240000e+09,6.820000e+08,7.497000e+09,0.0,0.0,7.497000e+09,1.241900e+10,2.535000e+09,-1.451000e+09,NaN,-2.130000e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0000002488-22-000078,,4,20220331,USD,6.691500e+10,1.336900e+10,5.354600e+10,1.158200e+10,5.581000e+09,6.001000e+09,5.533300e+10,0.0,0.0,5.533300e+10,6.691500e+10,4.740000e+09,-6.650000e+08,NaN,-9.410000e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0000002488-22-000123,,4,20220630,USD,6.750200e+10,1.346200e+10,5.404000e+10,1.233300e+10,5.523000e+09,6.810000e+09,5.516900e+10,0.0,0.0,5.516900e+10,6.750200e+10,4.964000e+09,-2.180000e+08,NaN,-1.893000e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0000002488-22-000170,,4,20220930,USD,6.781100e+10,1.442000e+10,5.339100e+10,1.326900e+10,6.691000e+09,6.578000e+09,5.454200e+10,0.0,0.0,5.454200e+10,6.781100e+10,3.398000e+09,-1.520000e+08,NaN,-2.815000e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0000002969-22-000010,,5,20211231,USD,2.712530e+10,6.483500e+09,2.064180e+10,1.274970e+10,2.630100e+09,1.011960e+10,1.437560e+10,0.0,0.0,1.437560e+10,2.712530e+10,2.953700e+09,1.590520e+10,NaN,-1.989200e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0000002969-22-000026,,5,20220331,USD,2.744970e+10,6.249400e+09,2.120030e+10,1.293900e+10,3.205100e+09,9.733900e+09,1.451070e+10,0.0,0.0,1.451070e+10,2.744970e+10,2.348700e+09,1.607590e+10,NaN,-1.985400e+09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


content
- goal
- problems / examples
  -missing liabilities and lieabilitiesnoncurrent tags
- what does the standardizer to
 - pre steps
   - deduplication
   - correct errors
 - main rules
  - iterateed
 - post rule
 - validation
- loading and saving information

-Example BalanceSheet
- input 
   -> filtered data 
   -> only statement, only one currency, only main company, only standardized tags
- what kind of logs are produced, what do they show
- what you should consider if using standardized information
  -> check logs / check validation summary 
- limitations
  wrong data, missing tags in data
  none standardized tags
  
- Empirical approach, definition is not taken into account